In [48]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [50]:
df = pd.read_csv("data/bmi.csv")
df.pop("Index")

columns = df.columns.values
rand = random.randint(3, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Gender': 'Female', 'Height': 195, 'Weight': 104}

In [51]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [52]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [53]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://bmi-production-54d6.up.railway.app/v1/models/cc-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0]
    # Menentukan angka terbesar dalam list menggunakan fungsi max()
    prediksi_terdekat = max(prediction)

    # Menentukan indeks dari angka terbesar menggunakan metode index()
    indeks = prediction.index(prediksi_terdekat)
    if indeks ==0:
        print('Extremely Weak')
    elif indeks ==1:
        print('Weak')
    elif indeks==2:
        print('Normal')
    elif indeks==3:
        print('Overweight')
    elif indeks==4:
        print('Obesity')
    elif indeks==5:
        print('Extreme Obesity')
    # Menampilkan hasil
    print("dengan probability:", prediksi_terdekat)
    print("Indeks ke:", indeks)

In [55]:
make_predictions(inputs)

Obesity
dengan probability: 0.177263692
Indeks ke: 4
